# Step 1: Load the Data

**Project Description**: This project aims to analyze which words in news headlines generate the most engagement. Headlines are from the r/news subreddit. 

In [1]:
import pandas as pd
import requests
import time

We start by pulling the last 10100 posts via the reddit API. We get our client id and secret key from https://www.reddit.com/prefs/apps. These are needed to authorize our access to the API. 

In [2]:
CLIENT_ID = 'Is52RILyqHIGQxgMFaN9eg'
SECRET_KEY = 'cA-ZFnUf7-p1u59uyZMiG_7fTpXQJA'

In [3]:
#Use CLIENT_ID and SECRET_KEY to get autharization to use Rfeddit API. 
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

data = {
    'grant_type': 'password',
    'username': 'PapaTheButt',
    'password': 'Pedersen83'
} #This password is no longer in use, no privacy violation possible. 

headers = {'User-Agent': 'MyAPI/0.0.1'}

res  = requests.post('https://www.reddit.com/api/v1/access_token', # 'access_token' found by looking at contents of res.json()
                    auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

#oauth is used to authenticate your reddit user when outside of reddit. 

<Response [200]>

In [5]:
# Make a request for the most recent posts in /r/news
res = requests.get("https://oauth.reddit.com/r/news/new",
                   headers=headers, params={'limit': '100'})

# Create dataframe to append to
df = pd.DataFrame() 
params = {'limit': '100'}

# Repeat inner loop 101 times, to get 10100 observations
for i in range(101):
    res = requests.get("https://oauth.reddit.com/r/news/new",
                   headers=headers, params=params)
    # loop through each post
    for post in res.json()['data']['children']:
        
        # Append to dataframe
        df = df.append({
            'Title': post['data']['title'],
            'Upvote_ratio': post['data']['upvote_ratio'],
            'Upvotes': post['data']['ups'],
            'Comments': post['data']['num_comments']
        }, ignore_index=True)
        kind = post['kind']
        id_ = post['data']['id']
    
    fullname = kind + ' ' + id_ # This variable is used to determine from what point on we will collect our data 
    params['after'] = fullname # We want to collect data "after" (which means before in the reddit API) that point
    time.sleep(2) #You can only make 30 requests per minute, that means 1 per 2 seconds
    print(i) # Just to make sure that our cell is actually running-- Scroll down to see dataframe
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


,Title,Upvote_ratio,Upvotes,Comments
0,Cash-strapped Britons give up pets as living c...,0.83,4.0,0.0
1,Anne Heche ‘peacefully taken off life support’...,0.92,11.0,0.0
2,Billionaires are funding a massive mineral dep...,0.87,11.0,6.0
3,China new bank loans tumble more than expected...,0.82,14.0,1.0
4,Risks of disaster at nuclear plant ‘increasing...,0.86,20.0,3.0
...,...,...,...,...
10095,Anne Heche “Not Expected To Survive” After Sev...,0.94,5474.0,1416.0
10096,"Over 900,000 Brazilians sign letter in defense...",0.96,1330.0,53.0
10097,2 detained after Brentwood 24 Hour Fitness sho...,0.89,200.0,32.0
10098,FBI searched Trump’s home to look for nuclear ...,0.91,39124.0,5365.0


Now we can safely save our data to a csv file, so that we can work with it in the next step. 

In [7]:
titles = df.to_csv('Titles.csv')